In [1]:
import pandas as pd
import numpy as np
import spacy
import gensim
import os
import codecs
import json
import re
from bs4 import BeautifulSoup

from gensim.corpora import Dictionary, MmCorpus
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.phrases import Phraser
import pyLDAvis
import pyLDAvis.gensim
import warnings
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('whitegrid')

Using TensorFlow backend.


In [ ]:
restaurant_ids1 = []

# open the businesses file
with open('./data/yelp_academic_dataset_business.json', encoding='utf_8') as f:
    
    # iterate through each line (json record) in the file
    for line in f:
        
        # convert the json record to a Python dict
        business = json.loads(line)
        
        # if this business is not a restaurant, skip to the next one
        if business["categories"] is not None and "Restaurants" in business["categories"]:
            # add the restaurant business id to our restaurant_ids set
            restaurant_ids1.append({'id':business['business_id'], 'city':business['city'], 
                                    'state':business['state'], 'categories':business['categories']})
        else:
            continue


In [ ]:
business

In [ ]:
restaurant_ids1

In [ ]:
from collections import Counter

In [ ]:
Counter([a for item in restaurant_ids1 for a in item['categories']]).most_common(10)

In [ ]:
restaurant_ids1[0].keys()

In [ ]:
# restaurants = [{key: item[key]} for item in restaurant_ids1 for key in item.keys() if key in ['categories', 'id']]

In [ ]:
%%time 

restaurants = []
for item in restaurant_ids1:
    tmp ={}
    for key, val in item.items():
        if key in ['categories', 'id']:
          tmp[key] = val
    restaurants.append(tmp)    

In [ ]:
restaurants[0:2]

In [ ]:
review_json_filepath = os.path.join('./data/',
                                    'yelp_academic_dataset_review.json')
ids = [item['id'] for item in restaurant_ids1]

rest_sample = np.random.choice(ids, 10000)

rests = []
with codecs.open(review_json_filepath, encoding='utf_8') as f:
    for line in f:
        review = json.loads(line)
        if review['business_id'] in rest_sample:
            rests.append(review)
        else:
            continue
            

In [ ]:
# print the number of unique restaurant ids in the dataset

print('{:,}'.format(len(restaurant_ids1)), u'restaurants in the dataset.')

# checking overlap in two methods
missing = []
with open('./data/yelp_academic_dataset_business.json', encoding='utf_8') as f:
    
    # iterate through each line (json record) in the file
    for line in f:
        business = json.loads(line)
        if business['business_id'] in diff:
            missing.append(business)
            
df = pd.DataFrame(missing)
            

In [ ]:
print(restaurants[0])
rests[0]


In [ ]:
%%time

combined = []
for item in rests:
    for listing in restaurants:
        if item['business_id']==listing['id']:
            item['category'] = listing['categories']
            combined.append(item)

In [ ]:
go = False
if go == True:
    with open('./data/processed.txt', 'w') as file:
        for item in combined:
            file.write(json.dumps(item) +'\n')

In [2]:
# load text
rests = []
with open('./processed/processed.txt', 'r') as file:
    for line in file:
        rests.append(json.loads(line))

In [4]:
from textblob import TextBlob

def sentiment(question):
    
    pol_sub = [TextBlob(question).sentiment.polarity, 
               TextBlob(question).sentiment.subjectivity]
    return pol_sub           


In [5]:
sentiment(rests[300]['text'])

[0.089088713219148, 0.5837922705314009]

In [12]:
sentiment(' '.join(reviews_cleaned[300]))

NameError: name 'reviews_cleaned' is not defined

In [32]:
len(rests[300])

11

In [37]:
 nlp(rests[300]['text'])[0:10]

A one-star blah. This place will be

In [43]:
rests[300]['text']

"A one-star blah. This place will be closed down and forgotten before the coffee is served. Let me premise this by saying I'm not one of those usual review whiners that can't find anything good about anywhere. La Coupole was simply disappointing. \n\nWhen we walked in, I thought that it would be such a fine, impressive place to bring out-of-town, business guests. However, the food and service did not live up to the standards of the atmosphere and the hotel it's attached to. \n\nWe went as a group of about twelve guests for a business lunch today. A few of us ordered the catch of the day, which was the red snapper. However, it was so flavourless and tough that it tasted like the catch of yesterday. The parsnips that came with it were tasty though. Two people ordered the hanger steak, which presented beautifully and looked delicious, but they both said their meat was cold. \nSome of the waitstaff were nice, one seemed very inexperienced and a couple of others seemed snooty and apathetic.

In [45]:
sentiment(' '.join(clean_text(rests[300]['text']))[0:100])

[-0.15555555555555559, 0.2888888888888889]

In [47]:
sentiment(' '.join(clean_text(rests[300]['text'])))

[-0.03305555555555556, 0.5113888888888889]

In [41]:
sentiment(' '.join(clean_text(rests[300]['text'])))

[0.07442520442520444, 0.5651851851851851]

In [51]:
rests[450]

{'business_id': '5RZt_9lrBSNDuzCyV1zPMw',
 'category': ['Fish & Chips', 'Restaurants', 'Southern', 'Seafood'],
 'cool': 1,
 'date': '2013-03-26',
 'funny': 0,
 'review_id': 'zKsJsZ-UlU45UDdFacAXtQ',
 'stars': 4,
 'text': "All I can say is, SO DELICIOUS!  I ordered the lunch special with the shrimp po'boy, fries, and a drink for around $6.50.  I took it to go, and it was still piping hot when I got back home 15 minutes later.  When I opened the container, fries came tumbling out, it was packed FULL!  I set them aside, and unwrapped the po'boy.  It was full of 3 large shrimp, fried crispy golden brown, lettuce, tomato, pickles, and sauce.  The bread was soft and fresh.  I added a little Louisiana Hot sauce and dug in!  It was amazing!  Such fresh flavors.  The fries were done perfectly, crisp on the outside, moist on the inside.  The iced tea is fresh brewed and nice and strong!  \nDon't let the location keep you away.  The restaurant was immaculately clean, well lit with ample seating. 

In [6]:
categories = [business['category'] for business in rests]

In [7]:
len(categories)

490049

In [8]:
reviews = pd.DataFrame(rests)

In [9]:
unique_labels = set([item for category in categories for item in category])

In [10]:
unique_labels

{'Acai Bowls',
 'Accountants',
 'Active Life',
 'Adult',
 'Adult Entertainment',
 'Afghan',
 'African',
 'Airport Lounges',
 'Airport Shuttles',
 'Airports',
 'Amateur Sports Teams',
 'American (New)',
 'American (Traditional)',
 'Amusement Parks',
 'Apartments',
 'Appliances',
 'Aquarium Services',
 'Aquariums',
 'Arabian',
 'Arcades',
 'Argentine',
 'Armenian',
 'Art Galleries',
 'Art Schools',
 'Arts & Crafts',
 'Arts & Entertainment',
 'Asian Fusion',
 'Australian',
 'Austrian',
 'Automotive',
 'Bagels',
 'Bakeries',
 'Bangladeshi',
 'Banks & Credit Unions',
 'Barbeque',
 'Barbers',
 'Bars',
 'Bartenders',
 'Basque',
 'Bavarian',
 'Beach Bars',
 'Beauty & Spas',
 'Bed & Breakfast',
 'Beer',
 'Beer Bar',
 'Beer Garden',
 'Beer Gardens',
 'Beer Hall',
 'Belgian',
 'Bike Repair/Maintenance',
 'Bikes',
 'Bistros',
 'Books',
 'Bookstores',
 'Botanical Gardens',
 'Bowling',
 'Brasseries',
 'Brazilian',
 'Breakfast & Brunch',
 'Breweries',
 'British',
 'Bubble Tea',
 'Buffets',
 'Building

In [11]:
unique_data = [list(x) for x in set(tuple(x) for x in categories)]


In [12]:
len(unique_data)

4953

In [13]:
reviews.shape

(490049, 11)

In [11]:
nlp = spacy.load('en')

In [44]:
def clean_text(text):
    
    # Remove HTML
    text = BeautifulSoup(text, "html.parser").get_text()
    
    #removes numbers, symbols, nextline symbols 
    
    text = re.sub("[^a-zA-Z'0-9]"," ", str(text))
    text = text.replace("\n", '').replace('  ', ' ').lower().strip('.')
    text = nlp(text)
    
    # removes punctuation and pronouns, returns lemmatized word
    lex = nlp.vocab['not']
    lex.is_stop = False
    
    words=[]
    for word in text:
        if word.pos_ != 'PUNCT' and not word.is_space and word.lemma_ !='-PRON-':
            words.append(word.lemma_)

    
    unigrams = [word for word in words if word not in ['t', 's', "v", "'s" "n't", 'food'
                                                      ]
               ]
    if len(unigrams) > 200:
        unigrams = unigrams[0:50] + unigrams[-50:]
    return unigrams

## Phrase modeling

In [16]:
text = reviews['text'][5]
samp_text = re.sub("[^a-zA-Z' ]", " ", str(text))
samp_text = samp_text.replace("\n\n", '').lower().strip().replace('  ', ' ').split('. ')
print(text)
samp_text


THE CLASSIC BURGER IS AMAZING.  It's definitely a top ten burger. The fries are good too.


["the classic burger is amazing  it's definitely a top ten burger the fries are good too"]

In [35]:
test = nlp(rests[300]['text'])
print(pd.DataFrame([(word, word.pos_, word.tag_, word.lemma_, word.orth_) for word in test], columns = ['word', 'pos', 'tag', 'lemma', 'orth']))

             word    pos   tag      lemma          orth
0               A    DET    DT          a             A
1             one    NUM    CD        one           one
2               -  PUNCT  HYPH          -             -
3            star   NOUN    NN       star          star
4            blah   NOUN    NN       blah          blah
5               .  PUNCT     .          .             .
6            This    DET    DT       this          This
7           place   NOUN    NN      place         place
8            will   VERB    MD       will          will
9              be   VERB    VB         be            be
10         closed   VERB   VBN      close        closed
11           down    ADV    RB       down          down
12            and  CCONJ    CC        and           and
13      forgotten   VERB   VBN     forget     forgotten
14         before    ADP    IN     before        before
15            the    DET    DT        the           the
16         coffee   NOUN    NN     coffee       

In [ ]:
print(clean_text(reviews['text'][5]))

['classic', 'burger', 'amazing', 'be', 'definitely', 'burger', 'fry', 'good']


In [50]:
%%time

reviews_cleaned = [clean_text(review['text']) for review in rests]
#reviews_cleaned = reviews['text'].apply(clean_text)
len(reviews_cleaned)

CPU times: user 50min 40s, sys: 3.75 s, total: 50min 44s
Wall time: 50min 45s


In [52]:
go = True
if go == True:
    with open("reviews_cleaned.txt", "w") as f:
        for line in reviews_cleaned:
            f.write(json.dumps(line) + "\n")

In [55]:
sentiment(' '.join(reviews_cleaned[5]))

[0.3933333333333333, 0.4333333333333334]

In [59]:
rests[5]['stars']

5

In [57]:
sentiment(rests[5]['text'])

[0.3933333333333333, 0.4333333333333334]

In [2]:
## start from here for next run
with open("./processed/stuff.txt", "r") as test:
    reviews_cleaned =[]
    for line in test:
        reviews_cleaned.append(json.loads(line))

In [8]:
from gensim.models import Phrases

#unigrams = [review.split(' ') for review in reviews_cleaned]
unigrams = reviews_cleaned
reviews_cleaned[0]

['place',
 'area',
 'staple',
 'year',
 'not',
 'change',
 'good',
 'thing',
 'stable',
 'reliable',
 'family',
 'go',
 'year',
 'st',
 'patty',
 'day',
 'corn',
 'beef',
 'nice',
 'place',
 'bar',
 'night',
 'dinner',
 'catch',
 'friend',
 'drink']

In [9]:
len(unigrams)

490049

In [10]:
bigrams = gensim.models.phrases.Phraser(Phrases(unigrams))

In [11]:
bigram_sents = []
for sent in unigrams:
   bigram_sents.append(bigrams[sent])

In [12]:
trigrams = Phrases(bigram_sents)

In [13]:

trigram_sents =''
for bigram in bigram_sents:
    trigram_sents += ' '.join(trigrams[bigram])

/home/ryeyoo/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:315: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [ ]:
#span = re.search('mastro_', trigram_sents).span()
#print(span)

In [ ]:
#trigram_sents[span[0]:span[1]+500]

In [14]:
trigram_phrases = [trigrams[bigram] for bigram in bigram_sents]

/home/ryeyoo/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:315: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [19]:
import re

multigram_list = re.findall(r'\w*_\w*_\w*', trigram_sents)

In [20]:
from collections import Counter

Counter(multigram_list).most_common(20)

[('sweet_potato_fry', 3472),
 ('mac_n_cheese', 2400),
 ('seat_right_away', 1838),
 ('http_www_yelp_com', 1167),
 ('french_onion_soup', 1075),
 ('hot_sour_soup', 995),
 ('corn_beef_hash', 948),
 ('pull_pork_sandwich', 868),
 ('soft_shell_crab', 786),
 ('staff_friendly_helpful', 695),
 ('lobster_mac_cheese', 693),
 ('vanilla_ice_cream', 594),
 ('staff_friendly_attentive', 589),
 ('creme_br_l_e', 579),
 ('beef_noodle_soup', 574),
 ('chicken_tikka_masala', 566),
 ('pico_de_gallo', 539),
 ('sweet_sour_chicken', 533),
 ('thin_crust_pizza', 530),
 ('thai_iced_tea', 519)]

In [17]:
trigram_sents[0:300]

'place area staple year not change good thing stable reliable family go year st_patty day corn_beef nice place bar night dinner catch friend drinkget mojo have appetite teaser love lpw frill bite eatnot decor staff_friendly fried_pickle tasty good valuebelieve award_star bear_mind type restaurant rev'

In [18]:
Counter([word for line in trigram_phrases for word in line]).most_common(20)

[('not', 735793),
 ('good', 475121),
 ('be', 356141),
 ('place', 330352),
 ('order', 249294),
 ('great', 229119),
 ('like', 213606),
 ('come', 211187),
 ('time', 197166),
 ('service', 185463),
 ('try', 152945),
 ('restaurant', 144547),
 ('get', 133677),
 ('go', 133172),
 ('eat', 127944),
 ('have', 123350),
 ('love', 110385),
 ('nice', 96988),
 ('menu', 95375),
 ('want', 87835)]

## Topic modeling

In [21]:
trigram_dictionary = Dictionary(trigram_phrases)
print(trigram_dictionary)
#print(trigram_dictionary.token2id)

Dictionary(214051 unique tokens: ['place', 'area', 'staple', 'year', 'not']...)


In [23]:
len(trigram_dictionary.keys())

214051

In [ ]:
# must give doc2bow the tokenized list from trigrams
corpus_yelp = [trigram_dictionary.doc2bow(phrase) for phrase in trigram_phrases]


In [ ]:
tfidf_yelp = models.TfidfModel(corpus_yelp)
tfidf_corpus = tfidf_yelp[corpus_yelp]

In [ ]:
%%time

#lda = LdaMulticore(corpus_yelp, num_topics = 20, id2word=trigram_dictionary, workers = 6, chunksize=10000, passes=2)

lda = gensim.models.ldamodel.LdaModel(corpus=tfidf_corpus, id2word=trigram_dictionary, num_topics=20, update_every=1, chunksize=10000, passes=2)


In [ ]:
lda.show_topics(num_topics=-1, num_words=4)

In [ ]:
def lda_description(review_text, min_topic_freq=0.05):
    """
    accept the original text of a review and (1) parse it with spaCy,
    (2) apply text pre-proccessing steps, (3) create a bag-of-words
    representation, (4) create an LDA representation, and
    (5) print a sorted list of the top topics in the LDA representation
    """
    
    # parse the review text with spaCy
    #parsed_review = nlp(review_text)
    
    # lemmatize the text and remove punctuation and whitespace
    unigram_review = clean_text(review_text)
    
    # apply the first-order and secord-order phrase models
    bigram_review = bigrams[unigram_review]
    trigram_review = trigrams[bigram_review]
    
    # remove any remaining stopwords
    trigram_review = [term for term in trigram_review
                      if not term in spacy.en.language_data.STOP_WORDS]
    
    # create a bag-of-words representation
    review_bow = trigram_dictionary.doc2bow(trigram_review)
    
    # create an LDA representation
    review_lda = lda[review_bow]
    
    # sort with the most highly related topics first
    review_lda = sorted(review_lda, key=lambda top_freq: (top_freq[0], top_freq[1]))
    
    for topic_number, freq in review_lda:
        if freq < min_topic_freq:
            break
            
        #print the most highly related topic names and frequencies
        #print ("{!s:25s}".format((topic_number, round(freq, 3))))
        print ((topic_number, round(freq, 3)))

In [ ]:
print(lda_description(reviews['text'][250]))
reviews['text'][250]

In [ ]:
lda.print_topic(9)
lda.print_topic(12)

In [ ]:
#lda.save('yelp_lda')

yelp_lda = gensim.models.ldamodel.LdaModel.load('yelp_lda')

In [ ]:
LDAvis_prepared = pyLDAvis.gensim.prepare(yelp_lda, tfidf_corpus,
                                              trigram_dictionary)
pyLDAvis.display(LDAvis_prepared)

In [ ]:
sns.barplot(x= reviews['stars'], y = reviews['stars'], estimator= lambda x: len(x)/len(reviews['stars']))
plt.ylabel('proportion')
plt.show()

sns.distplot(np.log(reviews['funny'].value_counts()))
plt.xlim(0,10)
plt.show()

In [ ]:
reviews.describe()

## Latent semantic analysis


In [ ]:
lsi = gensim.models.lsimodel.LsiModel(corpus=tfidf_corpus, id2word=trigram_dictionary, num_topics=250)


In [ ]:
lsi.print_topics(12, num_words = 4)

In [ ]:
print(trigram_phrases[10])

review = lsi[trigram_dictionary.doc2bow(trigram_phrases[10])]

sorted([topic for topic in review if topic[1] > .05], key = lambda topic: topic[1], reverse = True)[0:5]
      

## Word vectors

In [ ]:
w2vec = gensim.models.Word2Vec(trigram_phrases)

In [ ]:
w2vec.save("./models/review_vec")

In [ ]:
print(w2vec.similarity('france', 'america'))

w2vec.most_similar(positive=['lunch_special', 'bad'], negative=['burger'], topn=10)

In [ ]:
import rpy2.interactive as r

In [ ]:
%load_ext rmagic


In [1]:
from rpy2.robjects.packages import importr

# import R's "base" package
base = importr('base')

# import R's "utils" package
utils = importr('utils')

ImportError: libicuuc.so.54: cannot open shared object file: No such file or directory

In [ ]:
import keras